<a href="https://colab.research.google.com/github/alinaalborova/russian_idioms_processing/blob/main/IdiomSim_multiBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Idiom Paraphrase Experiments


Based on [Idiom Paraphrases: Seventh Heaven vs Cloud Nine](https://www.aclweb.org/anthology/W15-2709v2.pdf) by Maria Pershina Yifan He Ralph Grishman.


## Import Libraries

In [ ]:
!pip install transformers
!pip install tensor2tensor
!git clone https://github.com/jessevig/bertviz.git

In [ ]:
import numpy as np
import pandas as pd
from numpy import linalg as LA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import tensorflow as tf
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

!pip install annoy
from tqdm import tqdm_notebook
import networkx as nx       
from annoy import AnnoyIndex

     |████████████████████████████████| 655kB 3.0MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391621 sha256=a1d9333f76713adfda2f6d7359dcde6ed100b07db73dd0a584d2b8923b563494
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy


## Load Dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
dataset_dir = '/content/drive/MyDrive/ВКР/fedorov_clean.csv'
data = pd.read_csv(dataset_dir )
data.head()

,Unnamed: 0,Idiom,Definition
0,0,а вот на тебе!,Выражение удивления разочарования и т п
1,1,и вот на тебе!,Выражение удивления разочарования и т п
2,2,а был ли мальчик?,Было ли что либо в действительности на самом д...
3,3,а вот,Напротив наоборот
4,4,а вот и нет,Не выходит нельзя сделать то что намечается пр...


In [ ]:
data.shape

(14326, 3)

In [ ]:
data.drop('Unnamed: 0', axis=1, inplace=True)
data.drop(0, inplace=True)
data

,Idiom,Definition
1,и вот на тебе!,Выражение удивления разочарования и т п
2,а был ли мальчик?,Было ли что либо в действительности на самом д...
3,а вот,Напротив наоборот
4,а вот и нет,Не выходит нельзя сделать то что намечается пр...
5,вот и нет,Не выходит нельзя сделать то что намечается пр...
...,...,...
14321,ямская изба,В России XIII XVIII вв почтовая станция где пр...
14322,ясная голова,О том кто способен логично чётко мыслить
14323,яснее ясного,Совершенно понятно очевидно
14324,ясное дело,Ясно же конечно безусловно


In [ ]:
data.dropna(inplace=True)
data.shape

(13940, 2)

In [ ]:
data_test = data.sample(500, random_state=42)
data_test

,Idiom,Definition
11342,разводить тары-бары (растабары),Заниматься пустословием попусту болтать
10145,показать кукиш,То же что показывать дулю i кому А ты чай дума...
13580,устоять в слове,Выполнить обещанное
13289,точить балясы (лясы),Пустословить заниматься пустой болтовнёй Перво...
7802,не иметь духа (духу),Не отважиться не решиться что либо сказать сде...
...,...,...
10917,прищемлять хвост,Строго ограничивать кого либо в его действиях ...
6674,музейная редкость,Что либо давно уже вышедшее из употребления из...
10239,положение хуже губернаторского,Очень неприятное крайне затруднительное положе...
7938,не помнить себя,От волнения от каких либо переживаний не отдав...


## Get Embeddings

Definitions --> BERT --> Definitions embeddings

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.TFBertModel, ppb.BertTokenizer, 'bert-base-multilingual-cased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
BERT_model = model_class.from_pretrained(pretrained_weights)

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
BERT_model.summary()

Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  177853440 
Total params: 177,853,440
Trainable params: 177,853,440
Non-trainable params: 0
_________________________________________________________________


### Tokenize

In [ ]:
tokenized = data['Definition'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=50)))
#tokenized = data['Definition'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
#tokenized_i = data['Idiom'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True)))
print('Max sentence length: ', max([len(sen) for sen in tokenized]))
#print('Max idiom length: ', max([len(sen) for sen in tokenized_i]))

Max sentence length:  50


### Pad

In [ ]:
from keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length.
# 450 is chosen arbitrarily. It's slightly larger than the
# maximum training sentence length of 450.
MAX_LEN = 60

def pad_sentence(tokenized):
  print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
  print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

  # Pad our input tokens with value 0.
  # "post" indicates that we want to pad and truncate at the end of the sequence,
  # as opposed to the beginning.
  input_ids = pad_sequences(tokenized, maxlen=MAX_LEN, dtype="long", 
                            value=0, truncating="post", padding="post")
  print('\nDone.')
  return input_ids

padded = pad_sentence(tokenized)


Padding/truncating all sentences to 60 values...

Padding token: "[PAD]", ID: 0

Done.


In [ ]:
padded

array([[   101,  33703,  44325, ...,      0,      0,      0],
       [   101, 101673,  33190, ...,      0,      0,      0],
       [   101,  10778,  89461, ...,      0,      0,      0],
       ...,
       [   101,  36852,  32418, ...,      0,      0,      0],
       [   101,    540,  24905, ...,      0,      0,      0],
       [   101,  19597,  68140, ...,      0,      0,      0]])

In [ ]:
len(padded)

13940

### Mask

In [ ]:
def create_att_masks(input_ids):
  # Create attention masks
  attention_masks = []

  # For each sentence...
  for sent in input_ids:
      
      # Create the attention mask.
      #   - If a token ID is 0, then it's padding, set the mask to 0.
      #   - If a token ID is > 0, then it's a real token, set the mask to 1.
      att_mask = [int(token_id > 0) for token_id in sent]
      
      # Store the attention mask for this sentence.
      attention_masks.append(att_mask)
  attention_masks = np.array(attention_masks)    
  return attention_masks

masked = create_att_masks(padded)

In [ ]:
masked

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [ ]:
len(masked)

13940

### Embed

#### Batches

In [ ]:
def extract_full_embeddings(output):
  last_hidden_states = output[0] # lhs for all sentences
  extracted = []
  for i, el in enumerate(last_hidden_states): #for each sentence...
    extracted.append(last_hidden_states[i]) 
  return extracted

def use_batches(padded, masked):

  full_embeddings = []

  NUM_OF_IDIOMS_initial = len(masked)
  BATCH_SIZE = 500
  NUM_OF_IDIOMS = NUM_OF_IDIOMS_initial
  i = 0

  while NUM_OF_IDIOMS > 0:
    print(i)
    NUM_OF_IDIOMS -= BATCH_SIZE
    if i < NUM_OF_IDIOMS_initial - BATCH_SIZE:
      output_batch = BERT_model(padded[i:i+BATCH_SIZE], attention_mask = masked[i:i+BATCH_SIZE])
    else:
      output_batch = BERT_model(padded[i:NUM_OF_IDIOMS_initial], attention_mask = masked[i:NUM_OF_IDIOMS_initial])
    i += BATCH_SIZE
    embeddings_batch = extract_full_embeddings(output_batch)
    full_embeddings.append(embeddings_batch)

  full_embeddings_all = []
  for batch in full_embeddings:
    for sentence in batch:
      full_embeddings_all.append(sentence)

  return full_embeddings_all


In [ ]:

full_tokens = use_batches(padded, masked)
len(full_tokens)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500


13940

## Similarity Tricks

#### _Auxiliary_. Build TF-IDF

In [ ]:
tk = tf.keras.preprocessing.text.Tokenizer()
tk.fit_on_texts(data['Definition'])

tk_output = tk.sequences_to_matrix(tk.texts_to_sequences(data['Definition']), mode='tfidf')

In [ ]:
print(len(data['Definition']))
len(tk_output) == len(data['Definition'])

13940


True

In [ ]:
len(tk_output[0]) ## vocab size

20606

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
def align_tfidf(tk_output):
  weights_all = []
  for doc in tqdm_notebook(tk_output):
    weights = [float(0), ] # reserved for cls
    for w in doc:
      if w != float(0):
        weights.append(w)
    while len(weights) < MAX_LEN:
      weights.append(float(0))
    weights_all.append(weights)
  return weights_all

In [ ]:
tfidf_values = align_tfidf(tk_output)
len(tfidf_values)

13940

### Install Annoy & Define a Function

In [ ]:
def build_index(embeddings):
  f = 768
  Index = AnnoyIndex(f, 'angular')  # Length of item vector to be indexed

  for index, vector in enumerate(embeddings):
    Index.add_item(index, vector)
  Index.build(100)

  return Index

### Lexical Similarity

Compute cosine similarity between vectors Vd1 and Vd2, representing idiom descriptions d1 and d2 and weight each word in these vectors by its tf-idf score:  
$lexSim(d1, d2) = cosine(\vec{v}_{d1}, \vec{v}_{d2})$

where $\vec{v}$ d is a $\left |V \right |$-dimensional vector with $V$
being the vocabulary of all definition words.

In [ ]:
print(len(full_tokens[0][0]))
len(tfidf_values[0])

768


60

1. Get Vd1 and Vd2, representing idiom descriptions d1 and d2 and weight each word in these vectors by its tf-idf score

In [ ]:
def get_all_vd_values(tokens, tfidf_values):
  """
  Multplies each token vector in an idiom by its tf-idf value.
  """
  vd_values_total = []
  for id, full_token in enumerate(tokens): # for each sentence's tokens
    vd_values = []
    for i, el in enumerate(full_token):
      token_emb_by_tfidf = tf.math.multiply(el, tfidf_values[id][i])
      vd_values.append(token_emb_by_tfidf)
    vd_values_total.append(tf.math.reduce_mean(vd_values, axis=0))
  return vd_values_total

vd_values_all = get_all_vd_values(full_tokens, tfidf_values)
len(vd_values_all)

13940

In [ ]:
def get_max_dist(values):
  norms = []
  for value in values:
    norms.append(LA.norm(value))
  norms.sort()
  max_norms = norms[-2:]
  return max_norms[0] + max_norms[1]

MAX_DIST_LEXICAL = get_max_dist(vd_values_all)
MAX_DIST_LEXICAL

148.15347

In [ ]:
def get_vd_values(definition):
  '''
  idiom_tokens: list of strings
  '''
  idiom_id = list(data.Definition).index(definition)
  try:
    idiom_tokens = full_tokens[idiom_id]
  except:
    print(idiom_id)
  idiom_tfidf_values = tfidf_values[idiom_id]
  vd_values = []
  for i, el in enumerate(idiom_tokens):
    token_emb_by_tfidf = tf.math.multiply(el, idiom_tfidf_values[i])
    vd_values.append(token_emb_by_tfidf)  
  return tf.math.reduce_mean(vd_values, axis=0)

In [ ]:
id_vd = build_index(vd_values_all)

In [ ]:
def calculate_lex_sim(i1_id, i2_id):
  return 1 / (1 + id_vd.get_distance(i1_id, i2_id))

In [ ]:
calculate_lex_sim(list(data.Idiom).index('ахиллесова пята'), list(data.Idiom).index('больное место'))

0.5581202557519794

### Semantical Similarity

To capture the overall meaning of the definitions d we combine word embeddings
for all words in d using two combination schemes:
* Averaged sum:  
$\vec{averaged}_{d} = \frac{1}{\left |d  \right |}\sum_{word \in d} \vec{emb}(word)$
* Weighted sum:   
$\vec{weighted}_{d} = \frac{1}{\sum_{word \in d}tfidf_{word}}\sum_{word \in d}tfidf_{word} \cdot  \vec{emb}(word)$   
<br><br>

Then semantic similarity is measured as $semSim(d1,d2)= cosine(comb_{d1}, comb_{d2})$,  
where
$\vec{comb}_{d}$ is a 100-dimensional vector combined from word embeddings
$\vec{emb}(word)$ or words in description $d$ using either averaged or weighted combination
schemes. 


#### Averaged Sum

$\vec{averaged}_{d} = \frac{1}{\left |d  \right |}\sum_{word \in d} \vec{emb}(word)$

get average of all words in d

In [ ]:
avg_sums = []
## Since we have normalized sentence vecs (all lenghts are 60), we won't additionaly divide by sent lenght.
for definition in full_tokens:
  avg_sums.append(tf.math.reduce_sum(definition, axis=0))
print(len(avg_sums))
avg_sums[5][:10]

13940


<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([ 18.03615  ,  -5.0280895,  23.584335 ,  36.877872 ,   4.144063 ,
       -15.688772 , -18.595705 ,   4.65299  ,   4.878986 ,  20.066952 ],
      dtype=float32)>

##### Build Annoy Index

In [ ]:
id_avg = build_index(avg_sums)

In [ ]:
def get_avg_sum(definition_tokens):
  return tf.math.reduce_sum(definition_tokens, axis=0)

#### semSim

Semantic similarity is measured as $semSim(d1,d2)= cosine(comb_{d1}, comb_{d2})$,  
where
$\vec{comb}_{d}$ is a 100-dimensional vector combined from word embeddings
$\vec{emb}(word)$ or words in description $d$ using either averaged or weighted combination
schemes. 

In [ ]:
def calculate_sem_sim_avg(i1_id, i2_id):
  return 1 / (1 + id_avg.get_distance(i1_id, i2_id))

### IdiomSim

* lexSim captures the actual lexical overlap
* semSim can better capture the closeness in semantic meaning

To find an optimal balance between the two we consider their weighted sum:  
$IdiomSim(d1, d2) =  (1 - \alpha) \cdot  lexSim(d1, d2) + \alpha \cdot semSim(d1, d2)$
and decide on an $\alpha$ by optimizing for a maximal
F-score on a development dataset.

In [ ]:
#alpha = 0

## Using Average scheme for semantic similarity measurements
def calculate_idiom_sim_avg(idiom1, idiom2, alpha=0.5):
  i1_id = list(data.Idiom).index(idiom1)
  i2_id = list(data.Idiom).index(idiom2)
  #print('Lexical Similarity ', calculate_lex_sim(i1_id, i2_id))
  #print('Semantical Similarity ', calculate_sem_sim_avg(i1_id, i2_id))
  similarity_rate = (1 - alpha) * calculate_lex_sim(i1_id, i2_id) + alpha * calculate_sem_sim_avg(i1_id, i2_id)
  #print('Definition #1: {}\nDefinition #2: {}\nSimilarity rate:'.format(d1, d2, similarity_rate))
  return similarity_rate

In [ ]:
test = list(data.Idiom)[:500:11]

#### Find idioms similar to a given idiom

In [ ]:
from tqdm import tqdm

In [ ]:
def find_similar_idioms(idiom1, idioms, method='average', top_k=5):
  similarities = {}
  if method.lower() == 'average':
    print('Average method specified.')
    calculate_sim = calculate_idiom_sim_avg
  elif method.lower() == 'weighted':
    print('Weighted method specified.')
    calculate_sim = calculate_idiom_sim_weighted
  else:
    print('Incorrect method specified. Supported methods: average, weighted.\n'
    'Further calculations will use average method as default.')
    calculate_sim = calculate_idiom_sim_avg
  for id, idiom in enumerate(tqdm(idioms)):
    #print(id, idiom)
    if id < 13440:
      similarities[id] = calculate_sim(idiom1, idiom)
  print(similarities)
  sorted_dict = {}
  sorted_values = sorted(similarities.values())
  print(sorted_values)

  for i in tqdm(sorted_values):
    for k in similarities.keys():
      if similarities[k] == i:
        sorted_dict[k] = similarities[k]
        break

  print('\n\nIdioms similar to "{}":\n'.format(idiom1))
  print('{} method used.'.format(method))
  sorted_list = list(sorted_dict.items())
  sorted_list.reverse()
  for i, id in enumerate(sorted_list[1 :top_k + 1]):
    print('Idiom #{}: {}\n\tIts definition: {}\n\tSimilarity rate: {}'.
          format(i + 1, idioms[id[0]], list(data.Definition)[id[0]], sorted_dict[id[0]]))

## Test

In [ ]:
def find_similar_idioms_(idiom1, idioms, method='average', top_k=5):
  similarities = {}
  calculate_sim = calculate_idiom_sim_avg

  for id, idiom in enumerate(tqdm_notebook(idioms)):
    #print(id, idiom)
    if id < 13440:
      similarities[id] = calculate_sim(idiom1, idiom)
  sorted_dict = {}
  sorted_values = sorted(similarities.values())

  for i in sorted_values:
    for k in similarities.keys():
      if similarities[k] == i:
        sorted_dict[k] = similarities[k]
        break

  sorted_list = list(sorted_dict.items())
  sorted_list.reverse()
  candidates = []
  for i, id in enumerate(sorted_list[1 :top_k + 1]):
    candidates.append([idioms[id[0]], list(data.Definition)[id[0]]])
    
  return candidates

In [ ]:
test_size = [i for i in range(501)]
test_ranges = test_size[::10]
test_ranges

[0,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 210,
 220,
 230,
 240,
 250,
 260,
 270,
 280,
 290,
 300,
 310,
 320,
 330,
 340,
 350,
 360,
 370,
 380,
 390,
 400,
 410,
 420,
 430,
 440,
 450,
 460,
 470,
 480,
 490,
 500]

In [ ]:
test_ranges[:9]

[0, 10, 20, 30, 40, 50, 60, 70, 80]

In [ ]:
test_ranges[:11]

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [ ]:
ranges = [0, 1]

In [ ]:
rows_all = []
for r in test_ranges[18:22]:
  print(r, '/500_____________________\n')
  rows = []
  for i, idiom in enumerate(data_test.Idiom.values[r:r+10]):
    row = [[idiom, list(data.Definition)[list(data.Idiom).index(idiom)], ]]
    print(idiom, list(data.Definition)[list(data.Idiom).index(idiom)])
    candidates = find_similar_idioms_(idiom, list(data.Idiom))
    for candidate in candidates:
      row.append(candidate)
    rows.append(row)
  rows_all += rows

predictions = pd.DataFrame(rows_all)

180 /500_____________________

обойтись в копеечку Вызвать большие денежные затраты 
Average method specified.



залезть в чужую шкуру Ставить себя в положение кого либо 
Average method specified.



грешный человек Выражение признания своей слабости неправоты ошибки и т п 
Average method specified.



завей горе верёвочкой Словно ничего не случилось как ни в чём не бывало 
Average method specified.



стих находит Какое либо душевное состояние настроение полностью овладевает кем либо Кем либо овладевает желание совершить что либо нелепое сумасбродное 
Average method specified.


...


тридевятое царство p народно поэт p В русских сказках очень далёкая страна земля Шутл О каком либо очень отдалённом месте 
Average method specified.



лезть на рожон Начинать предпринимать что либо предельно рискованное обречённое на неудачу провал Действовать поспешно сгоряча не считаясь ни с чем не думая о последствиях 
Average method specified.



барашек в бумажке Взятка 
Average method specified.



свести к нулю Лишать всякого смысла значения 
Average method specified.



пуще глаза Очень предельно беречь кого либо или что либо 
Average method specified.


In [ ]:
predictions.head()

,0,1,2,3,4,5
0,"[разводить тары-бары (растабары), Заниматься п...","[поточить язык (языки), Заниматься пустой болт...","[воду качать, Заниматься чем либо бесполезным ...","[лазить по карманам, Заниматься карманным воро...","[воздавать честь хлебу-соли, Не отказываться о...","[смотреть по сторонам, Отвлекаться на пустяки ..."
1,"[показать кукиш, То же что показывать дулю i к...","[пялить зенки, p грубо прост p То же что пялит...","[качается от ветра, То же что без ветра качает...","[мухи не обидит, То же что комара не обидит i ...","[рука не поднимается, То же что рука не налега...","[выбросить на ветер, i Вижу зря я надеялся на ..."
2,"[устоять в слове, Выполнить обещанное ]","[держать слово, Выполнять обещанное ]","[закладывать за галстук, Выпивать напиваться п...","[мазнуть губы, Пообещать что либо но не выполн...","[мазать по губам, Дразнить пустыми обещаниями ...","[выйти из рамок, Выполнять больше чем предписа..."
3,"[точить балясы (лясы), Пустословить заниматься...","[всякое лыко в строку, Любая ошибка оплошность...","[ходить на цыпочках, Заискивать перед кем либо...","[не сводить глаз, i Во первых французы почиваю...","[бить баклуши, Праздно проводить время бездель...","[дым коромыслом, Шум гам суматоха и беспорядок..."
4,"[не иметь духа (духу), Не отважиться не решить...","[от корки до корки, От начала до конца ничего ...","[краем глаза, Невнимательно мельком не всматри...","[не открывать рот, Не пытаться выразить своё м...","[ни маковой росинки, Совсем ничего не сделать ...","[ни крошки, Совсем нисколько ничуть нет не ост..."


In [ ]:
predictions.to_csv('/content/drive/MyDrive/IdiomSim/predictions_multiBERT3.csv', encoding='utf-8-sig')